In [1]:
#LIBRARIES

In [2]:
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import animation

import numpy as np
import math
%matplotlib widget

import ipywidgets as widgets
from jupyros import ros3d
from ipywidgets import Button, ButtonStyle, FloatSlider, Layout, VBox, GridBox, HBox 

import jupyros as jr
import rospy

from nav_msgs.msg import Odometry
from geometry_msgs.msg import Twist
from std_msgs.msg import String
from rt2_assignment1.srv import Command, Velocity

from IPython.display import display

from tf import transformations

In [3]:
#DIRECTIONAL COMMANDS (MANUAL)

In [4]:
class Control_direction:
    
    def __init__(self, linear_slider, angular_slider):
        
        self.velocity_pub = rospy.Publisher('/cmd_vel', Twist, queue_size=1)
        self.twist = Twist()
        self.foward = 0.0
        self.turn = 0.0
        
        self.linear_slider = linear_slider
        self.angular_slider = angular_slider
        
    def stop(self):
        self.foward = 0.0
        self.turn = 0.0
    
    def forward(self):
        self.foward = 1.0
        self.turn = 0.0
        
    def backward(self):
        self.foward = -1.0
        self.turn = 0.0
    
    def CW(self):           #Clock wise
        self.foward = 0.0
        self.turn = 1.0
    
    def CCW(self):          #Counter clock wise
        self.foward = 0.0
        self.turn = -1.0
           
    def Twist_pub(self):
        self.twist.angular.z = self.turn*angular_slider.value
        self.twist.linear.x = self.foward*linear_slider.value        
        
        self.velocity_pub.publish(self.twist)

In [5]:
class Store_value(list):
    
    def store_data(self, data):
        super().store_data(data)
        if super().__len__() > self.max_size:
            super().pop(0)    
            
    def __init__(self, max_size, space=[]):
        super().__init__(space)
        self.max_size = max_size    

In [6]:
#GLOBAL VARIABLES

In [7]:
from enum import Enum
output = widgets.Output()

# Initial values for the velocities
linear_value = 0.5
angular_value = 0.5

# Variables to store values

odom_max_size = 1000
odom_max_x = Store_value(odom_max_size)
odom_max_y = Store_value(odom_max_size)
odom_max_rot = 0

vel_max_size = 100

vel_x_ = Store_value(vel_max_size, [linear_value]*vel_max_size)
odom_vel_x = Store_value(vel_max_size, [0]*vel_max_size)
odom_vel_w = Store_value(vel_max_size, [0]*vel_max_size)

t = np.linspace(0, 1, vel_max_size)

class RUN(Enum):
    STOP = 0
    START  = 1
    ARROW  = 2

run  = RUN.STOP
# /user_interface Client service
ui_client = None
vel_client = None

In [8]:
#SLIDER VELOCITY COMMAND (MANUAL)

In [9]:
max_vel_ = 2.0
linear_slider = widgets.FloatSlider(description = 'Linear Velocity')#, value = linear_value)
angular_slider = widgets.FloatSlider(description = 'Angular Velocity')#, value = angular_value)

VBox([linear_slider, angular_slider])

def change_linear_vel(vel):
    vel_client(vel["new"], odom_vel_w[-1])

def change_angular_vel(vel):
    vel_client(vel_x_[-1], vel["new"])
    
def printf(linear_slider, angular_slider):
	print('{}*{}'.format(linear_slider, angular_slider))

out1 = widgets.interactive_output(change_linear_vel, {'Linear Velocity': linear_slider})
out2 = widgets.interactive_output(change_angular_vel, {'Angular Velocity': angular_slider})

widgets.HBox([widgets.VBox([linear_slider]), out1])
widgets.HBox([widgets.VBox([angular_slider]), out2])

In [10]:
#START/STOP BUTTOM COMMAND (MANUAL)

In [11]:
button_stop  = Button(description='STOP',
               layout=Layout(width='auto', align="center", grid_area='button_stop'),
               style=ButtonStyle(button_color='red'))

button_start  = Button(description='START',
                layout=Layout(width='auto', grid_area='button_start'),
                style=ButtonStyle(button_color='green'))

HBox([button_start, button_stop])

GridBox(children=[button_stop, button_start],
        layout=Layout(
        width='100%',
        grid_template_rows='auto auto',
        grid_template_columns='0% 25%',
        grid_template_areas='''
        ". button_stop ."
        ". button_start ."
        ''')
       )


GridBox(children=(Button(description='STOP', layout=Layout(grid_area='button_stop', width='auto'), style=Butto…

In [12]:
#DIRECTIONAL BUTTOM COMMANDS (MANUAL)

In [13]:
button_foward = Button(description='\u2191', #Unicode Character 'UPWARDS ARROW' (U+2191)
                 layout=Layout(width='auto', grid_area='button_foward'),                        
                 style=ButtonStyle(button_color='lightgray'))

button_backwards = Button(description='\u2193', #Unicode Character 'DOWNWARDS ARROW' (U+2193)
                 layout=Layout(width='auto', grid_area='button_backwards'),
                 style=ButtonStyle(button_color='lightgray'))

button_right = Button(description='\u2192', #Unicode Character 'RIGHTWARDS ARROW' (U+2192)
                 layout=Layout(width='auto', grid_area='button_right'),
                 style=ButtonStyle(button_color='lightgray'))

button_left = Button(description='\u2190', #Unicode Character 'LEFTWARDS ARROW' (U+2190)
                 layout=Layout(width='auto', grid_area='button_left'),
                 style=ButtonStyle(button_color='lightgray'))

HBox([button_foward, button_backwards, button_left, button_right])

GridBox(children=[button_foward, button_backwards, button_left, button_right],
        layout=Layout(
        width='100%',
        grid_template_rows='auto auto auto',
        grid_template_columns='25% 25% 25%',
        grid_template_areas='''
        ". button_foward ."
        "button_left . button_right"
        ". button_backwards ."
        ''')
       )

GridBox(children=(Button(description='↑', layout=Layout(grid_area='button_foward', width='auto'), style=Button…

In [14]:
# CALLBACK FUNCTIONS

In [18]:
pad = Control_direction(linear_slider, angular_slider) 
    
def robot_stop(buttom):

    global run
    if run ==  RUN.START:
        ui_client("stop")
    elif run == RUN.ARROW:
        pad.stop()
        pad.pubTwist()
    
    run = RUN.STOP
    
def robot_start(buttom):

    global run
    if run != RUN.START:
        ui_client("start")
        run = RUN.START        
        
def command_buttons(buttom):

    global run

    if run == RUN.START:
        robot_stop(buttom)
        
    if buttom.layout.grid_area == 'button_foward':
        pad.forward()
    elif buttom.layout.grid_area == 'button_backwards':
        pad.backward()
    elif buttom.layout.grid_area == 'button_right':
        pad.turnClock()
    elif buttom.layout.grid_area == 'button_left':
        pad.turnCounter()
    else:
        return
    
    run = RUN.ARROW    

def odometry_callback(msg):
    global odom_max_rot, odom_max_x, odom_max_y
    
    global run
    
    odom_max_x.store_data(msg.pose.pose.position.x)
    odom_max_y.store_data(msg.pose.pose.position.y)
    
    quaternion = (
        msg.pose.pose.orientation.x,
        msg.pose.pose.orientation.y,
        msg.pose.pose.orientation.z,
        msg.pose.pose.orientation.w)
    euler = transformations.euler_from_quaternion(quaternion)
    odom_max_rot = euler[2]
    
    vel_x_.store_data(linear_slider.value)
    odom_vel_x.store_data(msg.twist.twist.linear.x)
    
    odom_vel_w.store_data(angular_slider.value)
    vel_w_odm.store_data(msg.twist.twist.angular.z)
    
    if run == RUN.ARROW:
        pad.pubTwist()

In [19]:
#BUTTONS COMMUNICATION

In [20]:
button_start.on_click(robot_start)
button_foward.on_click(command_buttons)
button_backwards.on_click(command_buttons)
button_right.on_click(command_buttons)
button_left.on_click(command_buttons)

In [21]:
#ROS NODE

In [ ]:
rospy.init_node("jupyter_user_interface")

sub = rospy.Subscriber('/odom', Odometry, odometry_callback)

rospy.wait_for_service('/vel')
vel_client = rospy.ServiceProxy('/vel', Velocity)

rospy.wait_for_service('/user_interface')
ui_client = rospy.ServiceProxy('/user_interface', Command)

In [ ]:
#ROS ACTION

In [ ]:
from rt2_assignment1.msg import Assignment1ActionResult

goal = {'reached':0, 'canceled':0}
time = [] #empty

def goal_callback(msg):
    global goal
    
    if msg.result.result:
        goal['reached'] = goal['reached']++
        duration = msg.header.stamp - msg.status.goal_id.stamp
        time.store_data(duration.to_sec())
    else:
        goal['canceled'] = goal['canceled']++

goal_result = rospy.Subscriber('/go_to_point/result', Assignment1ActionResult, goal_callback)